In [2]:
%load_ext rpy2.ipython

In [3]:
%%R
set.seed(88888888) # maximum luck

library(DESeq2)
library(plotly)
library(ggplot2)
library(viridis)
library(magrittr)
library(pheatmap)
library(DescTools)
library(pdfCluster)
library(RColorBrewer)
library(SummarizedExperiment)
library(caret)
library(class)
library(htmlwidgets)

Loading required package: S4Vectors
Loading required package: stats4
Loading required package: BiocGenerics

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, intersect, is.unsorted, lapply, Map, mapply,
    match, mget, order, paste, pmax, pmax.int, pmin, pmin.int,
    Position, rank, rbind, Reduce, rownames, sapply, setdiff, sort,
    table, tapply, union, unique, unsplit, which.max, which.min


Attaching package: ‘S4Vectors’

The following object is masked from ‘package:utils’:

    findMatches

The following objects are masked from ‘package:base’:

    expand.grid, I, unname

Loading required package: IRanges
Loading required package: GenomicRanges
Loading required package: GenomeInfoDb
Loading require

In [4]:
%%R
OUT_DIR <- "/mnt/home/yuankeji/RanceLab/reticula_new/reticula/data/tcdd/output/"
N_FOLDS <- 10

In [5]:
%%R
vst.counts <- readRDS(paste(OUT_DIR, "vst_counts0.01vs0_time_cross.Rds", sep = ""))
print(vst.counts)

class: DESeqTransform 
dim: 9297 91 
metadata(1): version
assays(1): ''
rownames(9297): ENSMUSG00000000001 ENSMUSG00000000028 ...
  ENSMUSG00000118462 ENSMUSG00000118541
rowData names(4): baseMean baseVar allZero dispFit
colnames(91): SRR7817611 SRR7817612 ... SRR4317615 SRR4317616
colData names(3): Sample Dose sizeFactor


In [6]:
%%R
sample <- colData(vst.counts)
# print(sample)

In [7]:
%%R
vst.count.mtx <-
   vst.counts %>% SummarizedExperiment::assay() %>% as.data.frame()
print(length(vst.count.mtx))
 head(vst.count.mtx)

[1] 91
                   SRR7817611 SRR7817612 SRR7817614 SRR7817615 SRR7817616
ENSMUSG00000000001  15.595318  15.614104   15.55178  15.627745  15.601970
ENSMUSG00000000028   9.915932   9.776246   10.01617   9.866338   9.934504
ENSMUSG00000000049  19.743729  19.723414   19.97462  19.944275  19.947717
ENSMUSG00000000058  10.500222  10.636379   10.66667  10.388602  10.908824
ENSMUSG00000000085  12.209777  11.972662   12.11671  12.117091  11.772576
ENSMUSG00000000088  16.576893  16.582078   16.54791  16.265580  16.221265
                   SRR7817623 SRR7817624 SRR7817625 SRR7817626 SRR7817627
ENSMUSG00000000001   15.53233  15.555347  15.702149  15.667955   15.50766
ENSMUSG00000000028   10.09052   9.906087   9.804369   9.960843   10.47578
ENSMUSG00000000049   19.96450  19.933932  19.874605  19.873780   19.89949
ENSMUSG00000000058   10.65414  11.008310  10.430383  10.589063   10.55120
ENSMUSG00000000085   11.93432  12.188878  12.285903  12.125150   11.82877
ENSMUSG00000000088   16.17851  

ENSMUSG00000000085   12.22528   12.10583   12.22505   12.11853   12.15772
ENSMUSG00000000088   16.17465   16.20538   15.63263   15.66697   15.05741
                   SRR4317614 SRR4317617 SRR4317618 SRR4317607 SRR4317609
ENSMUSG00000000001   15.19008   15.34284   15.29818   14.98142   15.22135
ENSMUSG00000000028   10.25854   10.36374   10.12165   10.07093   10.33969
ENSMUSG00000000049   20.19524   20.23854   20.20695   20.09852   20.16060
ENSMUSG00000000058   10.90348   11.34593   11.04140   11.08147   11.22135
ENSMUSG00000000085   12.01708   12.24278   12.15252   12.26732   11.58821
ENSMUSG00000000088   15.14629   15.01634   14.95516   15.07132   14.77912
                   SRR4317610 SRR4317611 SRR4317612 SRR4317613 SRR4317615
ENSMUSG00000000001   15.32976   15.16554   15.05158   15.09623   15.14719
ENSMUSG00000000028   10.15961   10.42678   10.08590   10.23070   10.29295
ENSMUSG00000000049   20.17354   20.15730   20.15705   20.22385   20.11501
ENSMUSG00000000058   11.45440   11.026

In [8]:
%%R
tcdd_dose_detail.vec <-
   readRDS(paste(OUT_DIR, "tcdd_dose_detail_vec0.01vs0_time_cross.Rds", sep = ""))
print(tcdd_dose_detail.vec)

 [1] 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00
[16] 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00
[31] 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00
[46] 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.01 0.01 0.01 0.01 0.01 0.01
[61] 0.01 0.01 0.01 0.01 0.00 0.00 0.00 0.01 0.00 0.00 0.00 0.00 0.01 0.00 0.01
[76] 0.01 0.01 0.01 0.01 0.00 0.01 0.01 0.01 0.00 0.00 0.00 0.00 0.00 0.01 0.01
[91] 0.01


In [9]:
%%R
rxn2ensembls.nls <-
   readRDS(paste(OUT_DIR, "rxn2ensembls_nls0.01vs0_time_cross.Rds", sep = ""))
print(length(rxn2ensembls.nls))
print(rxn2ensembls.nls)

[1] 9825
$`R-MMU-170666`
 [1] "ENSMUSG00000000001" "ENSMUSG00000005580" "ENSMUSG00000020431"
 [4] "ENSMUSG00000020654" "ENSMUSG00000021536" "ENSMUSG00000022220"
 [7] "ENSMUSG00000022376" "ENSMUSG00000022840" "ENSMUSG00000022994"
[10] "ENSMUSG00000024524" "ENSMUSG00000028777" "ENSMUSG00000031659"
[13] "ENSMUSG00000032562" "ENSMUSG00000057614"

$`R-MMU-170671`
 [1] "ENSMUSG00000000001" "ENSMUSG00000005580" "ENSMUSG00000020431"
 [4] "ENSMUSG00000020654" "ENSMUSG00000021536" "ENSMUSG00000022220"
 [7] "ENSMUSG00000022376" "ENSMUSG00000022840" "ENSMUSG00000022994"
[10] "ENSMUSG00000024524" "ENSMUSG00000028777" "ENSMUSG00000031659"
[13] "ENSMUSG00000032562" "ENSMUSG00000057614"

$`R-MMU-170674`
 [1] "ENSMUSG00000000001" "ENSMUSG00000005580" "ENSMUSG00000020431"
 [4] "ENSMUSG00000020654" "ENSMUSG00000021536" "ENSMUSG00000022220"
 [7] "ENSMUSG00000022376" "ENSMUSG00000022840" "ENSMUSG00000022994"
[10] "ENSMUSG00000024524" "ENSMUSG00000028777" "ENSMUSG00000031659"
[13] "ENSMUSG00000032562" "ENSM

 [46] "ENSMUSG00000028777" "ENSMUSG00000028950" "ENSMUSG00000028971"
 [49] "ENSMUSG00000029064" "ENSMUSG00000029072" "ENSMUSG00000029236"
 [52] "ENSMUSG00000029371" "ENSMUSG00000029372" "ENSMUSG00000029373"
 [55] "ENSMUSG00000029379" "ENSMUSG00000029380" "ENSMUSG00000029417"
 [58] "ENSMUSG00000029530" "ENSMUSG00000029663" "ENSMUSG00000029713"
 [61] "ENSMUSG00000029819" "ENSMUSG00000031778" "ENSMUSG00000031861"
 [64] "ENSMUSG00000031980" "ENSMUSG00000032192" "ENSMUSG00000032562"
 [67] "ENSMUSG00000032766" "ENSMUSG00000033717" "ENSMUSG00000034117"
 [70] "ENSMUSG00000034855" "ENSMUSG00000035042" "ENSMUSG00000035383"
 [73] "ENSMUSG00000035431" "ENSMUSG00000035448" "ENSMUSG00000036353"
 [76] "ENSMUSG00000036362" "ENSMUSG00000036381" "ENSMUSG00000036402"
 [79] "ENSMUSG00000036437" "ENSMUSG00000036832" "ENSMUSG00000037010"
 [82] "ENSMUSG00000037014" "ENSMUSG00000037140" "ENSMUSG00000037346"
 [85] "ENSMUSG00000037944" "ENSMUSG00000038607" "ENSMUSG00000038668"
 [88] "ENSMUSG00000038811" "ENSMUS

 [4] "ENSMUSG00000019775" "ENSMUSG00000020599" "ENSMUSG00000021219"
 [7] "ENSMUSG00000024186" "ENSMUSG00000026357" "ENSMUSG00000026358"
[10] "ENSMUSG00000026475" "ENSMUSG00000026527" "ENSMUSG00000026678"
[13] "ENSMUSG00000028777" "ENSMUSG00000029101" "ENSMUSG00000032562"
[16] "ENSMUSG00000038530" "ENSMUSG00000042671" "ENSMUSG00000051079"
[19] "ENSMUSG00000052087" "ENSMUSG00000057614" "ENSMUSG00000059810"

$`R-MMU-8982019`
 [1] "ENSMUSG00000000001" "ENSMUSG00000002458" "ENSMUSG00000002459"
 [4] "ENSMUSG00000019775" "ENSMUSG00000020599" "ENSMUSG00000021219"
 [7] "ENSMUSG00000024186" "ENSMUSG00000026357" "ENSMUSG00000026358"
[10] "ENSMUSG00000026475" "ENSMUSG00000026527" "ENSMUSG00000026678"
[13] "ENSMUSG00000028777" "ENSMUSG00000029101" "ENSMUSG00000032562"
[16] "ENSMUSG00000038530" "ENSMUSG00000042671" "ENSMUSG00000051079"
[19] "ENSMUSG00000052087" "ENSMUSG00000057614" "ENSMUSG00000059810"

$`R-MMU-8982020`
 [1] "ENSMUSG00000000001" "ENSMUSG00000002458" "ENSMUSG00000002459"
 [4] "ENSMUS

 [1] "ENSMUSG00000000085" "ENSMUSG00000015120" "ENSMUSG00000018537"
 [4] "ENSMUSG00000024325" "ENSMUSG00000025577" "ENSMUSG00000025578"
 [7] "ENSMUSG00000026021" "ENSMUSG00000026484" "ENSMUSG00000026739"
[10] "ENSMUSG00000028796" "ENSMUSG00000037652" "ENSMUSG00000039989"
[13] "ENSMUSG00000040669"

$`R-MMU-4570463`
 [1] "ENSMUSG00000000085" "ENSMUSG00000015120" "ENSMUSG00000018537"
 [4] "ENSMUSG00000020265" "ENSMUSG00000020738" "ENSMUSG00000024325"
 [7] "ENSMUSG00000025577" "ENSMUSG00000025578" "ENSMUSG00000026484"
[10] "ENSMUSG00000026739" "ENSMUSG00000028796" "ENSMUSG00000031347"
[13] "ENSMUSG00000037652" "ENSMUSG00000039989" "ENSMUSG00000040669"

$`R-MMU-4570499`
 [1] "ENSMUSG00000000085" "ENSMUSG00000015120" "ENSMUSG00000018537"
 [4] "ENSMUSG00000020738" "ENSMUSG00000021546" "ENSMUSG00000024325"
 [7] "ENSMUSG00000025577" "ENSMUSG00000025578" "ENSMUSG00000026484"
[10] "ENSMUSG00000026739" "ENSMUSG00000028796" "ENSMUSG00000037652"
[13] "ENSMUSG00000039989" "ENSMUSG00000040669"

$`R-MM

 [1] "ENSMUSG00000000126" "ENSMUSG00000009900" "ENSMUSG00000010797"
 [4] "ENSMUSG00000012282" "ENSMUSG00000015957" "ENSMUSG00000018486"
 [7] "ENSMUSG00000021994" "ENSMUSG00000022382" "ENSMUSG00000022996"
[10] "ENSMUSG00000022997" "ENSMUSG00000026167" "ENSMUSG00000027840"
[13] "ENSMUSG00000028173" "ENSMUSG00000029671" "ENSMUSG00000030093"
[16] "ENSMUSG00000030170" "ENSMUSG00000033227" "ENSMUSG00000036856"
[19] "ENSMUSG00000036961"

$`R-MMU-3781943`
 [1] "ENSMUSG00000000126" "ENSMUSG00000009900" "ENSMUSG00000010797"
 [4] "ENSMUSG00000012282" "ENSMUSG00000015957" "ENSMUSG00000018486"
 [7] "ENSMUSG00000021994" "ENSMUSG00000022382" "ENSMUSG00000022996"
[10] "ENSMUSG00000022997" "ENSMUSG00000026167" "ENSMUSG00000027840"
[13] "ENSMUSG00000028173" "ENSMUSG00000029671" "ENSMUSG00000030093"
[16] "ENSMUSG00000030170" "ENSMUSG00000033227" "ENSMUSG00000036856"
[19] "ENSMUSG00000036961"

$`R-MMU-205328`
[1] "ENSMUSG00000000127" "ENSMUSG00000005672" "ENSMUSG00000019966"
[4] "ENSMUSG00000032113" "ENSM

...

 [7] "ENSMUSG00000060981" "ENSMUSG00000061482" "ENSMUSG00000064288"
[10] "ENSMUSG00000067455" "ENSMUSG00000069266" "ENSMUSG00000069274"
[13] "ENSMUSG00000069305" "ENSMUSG00000069306" "ENSMUSG00000091405"
[16] "ENSMUSG00000096010"

$`R-MMU-5651657`
 [1] "ENSMUSG00000045098" "ENSMUSG00000055067" "ENSMUSG00000059851"
 [4] "ENSMUSG00000060093" "ENSMUSG00000060639" "ENSMUSG00000060678"
 [7] "ENSMUSG00000060981" "ENSMUSG00000061482" "ENSMUSG00000064288"
[10] "ENSMUSG00000067455" "ENSMUSG00000069266" "ENSMUSG00000069274"
[13] "ENSMUSG00000069305" "ENSMUSG00000069306" "ENSMUSG00000091405"
[16] "ENSMUSG00000096010"

$`R-MMU-8855062`
[1] "ENSMUSG00000045100"

$`R-MMU-162683`
[1] "ENSMUSG00000045140"

$`R-MMU-2395873`
[1] "ENSMUSG00000045211"

$`R-MMU-2395879`
[1] "ENSMUSG00000045211"

$`R-MMU-2395965`
[1] "ENSMUSG00000045211"

$`R-MMU-444848`
[1] "ENSMUSG00000045232" "ENSMUSG00000060735"

$`R-MMU-444859`
[1] "ENSMUSG00000045232" "ENSMUSG00000049741" "ENSMUSG00000066090"

$`R-MMU-8874435`
[1] "EN

[1] "ENSMUSG00000050211" "ENSMUSG00000070719"

$`R-MMU-451360`
[1] "ENSMUSG00000050272" "ENSMUSG00000060534"

$`R-MMU-5682285`
[1] "ENSMUSG00000050296"

$`R-MMU-517536`
[1] "ENSMUSG00000050350"

$`R-MMU-191983`
[1] "ENSMUSG00000050370"

$`R-MMU-2161660`
[1] "ENSMUSG00000050737"

$`R-MMU-727740`
[1] "ENSMUSG00000050822"

$`R-MMU-727768`
[1] "ENSMUSG00000050822"

$`R-MMU-1483096`
[1] "ENSMUSG00000050860"

$`R-MMU-1483159`
[1] "ENSMUSG00000050860"

$`R-MMU-417842`
[1] "ENSMUSG00000050921"

$`R-MMU-429786`
[1] "ENSMUSG00000050931"

$`R-MMU-190782`
[1] "ENSMUSG00000050953"

$`R-MMU-191072`
[1] "ENSMUSG00000050953"

$`R-MMU-375340`
[1] "ENSMUSG00000051056"

$`R-MMU-174959`
[1] "ENSMUSG00000051147"

$`R-MMU-174963`
[1] "ENSMUSG00000051147"

$`R-MMU-8851679`
[1] "ENSMUSG00000051212"

$`R-MMU-444171`
[1] "ENSMUSG00000051314"

$`R-MMU-9831007`
[1] "ENSMUSG00000051367" "ENSMUSG00000074934"

$`R-MMU-8869694`
[1] "ENSMUSG00000051439"

$`R-MMU-2161651`
[1] "ENSMUSG00000051483"

$`R-MMU-9729538`
[1] 

[1] "ENSMUSG00000056880"

$`R-MMU-4754176`
 [1] "ENSMUSG00000056962" "ENSMUSG00000069265" "ENSMUSG00000069267"
 [4] "ENSMUSG00000069273" "ENSMUSG00000069310" "ENSMUSG00000074403"
 [7] "ENSMUSG00000081058" "ENSMUSG00000093769" "ENSMUSG00000099517"
[10] "ENSMUSG00000099583" "ENSMUSG00000100210" "ENSMUSG00000101355"
[13] "ENSMUSG00000101972"

$`R-MMU-5661122`
 [1] "ENSMUSG00000056962" "ENSMUSG00000069265" "ENSMUSG00000069267"
 [4] "ENSMUSG00000069273" "ENSMUSG00000069310" "ENSMUSG00000074403"
 [7] "ENSMUSG00000081058" "ENSMUSG00000093769" "ENSMUSG00000099517"
[10] "ENSMUSG00000099583" "ENSMUSG00000100210" "ENSMUSG00000101355"
[13] "ENSMUSG00000101972"

$`R-MMU-5661124`
 [1] "ENSMUSG00000056962" "ENSMUSG00000060093" "ENSMUSG00000060639"
 [4] "ENSMUSG00000060678" "ENSMUSG00000060981" "ENSMUSG00000061482"
 [7] "ENSMUSG00000064288" "ENSMUSG00000067455" "ENSMUSG00000069266"
[10] "ENSMUSG00000069274" "ENSMUSG00000069305" "ENSMUSG00000069306"
[13] "ENSMUSG00000091405" "ENSMUSG00000096010"

$`R-M

[1] "ENSMUSG00000061878"

$`R-MMU-428779`
[1] "ENSMUSG00000062064"

$`R-MMU-8987120`
[1] "ENSMUSG00000062157"

$`R-MMU-8937442`
[1] "ENSMUSG00000062181" "ENSMUSG00000069922"

$`R-MMU-1254376`
[1] "ENSMUSG00000062209"

$`R-MMU-211923`
[1] "ENSMUSG00000062432"

$`R-MMU-189423`
[1] "ENSMUSG00000062729"

$`R-MMU-6786421`
[1] "ENSMUSG00000062778"

$`R-MMU-109341`
[1] "ENSMUSG00000062908"

$`R-MMU-77338`
[1] "ENSMUSG00000062908"

$`R-MMU-77345`
[1] "ENSMUSG00000062908"

$`R-MMU-1237160`
[1] "ENSMUSG00000062937"

$`R-MMU-6810376`
[1] "ENSMUSG00000063049"

$`R-MMU-5674385`
[1] "ENSMUSG00000063065" "ENSMUSG00000063358"

$`R-MMU-9610166`
[1] "ENSMUSG00000063065" "ENSMUSG00000063358"

$`R-MMU-2408505`
[1] "ENSMUSG00000063179"

$`R-MMU-3446785`
[1] "ENSMUSG00000063235"

$`R-MMU-351947`
[1] "ENSMUSG00000063410"

$`R-MMU-6810076`
[1] "ENSMUSG00000063428"

$`R-MMU-70577`
[1] "ENSMUSG00000063445" "ENSMUSG00000076441"

$`R-MMU-3204311`
[1] "ENSMUSG00000063558"

$`R-MMU-8944220`
[1] "ENSMUSG00000063564"

[1] "ENSMUSG00000089943" "ENSMUSG00000090171"

$`R-MMU-9756134`
[1] "ENSMUSG00000089943" "ENSMUSG00000090171"

$`R-MMU-9756156`
[1] "ENSMUSG00000089943" "ENSMUSG00000090171"

$`R-MMU-9756183`
[1] "ENSMUSG00000089943" "ENSMUSG00000090171"

$`R-MMU-9632038`
[1] "ENSMUSG00000089960"

$`R-MMU-9632039`
[1] "ENSMUSG00000089960"

$`R-MMU-5695989`
[1] "ENSMUSG00000090150"

$`R-MMU-209821`
[1] "ENSMUSG00000093806"

$`R-MMU-191323`
[1] "ENSMUSG00000093930"

$`R-MMU-159790`
[1] "ENSMUSG00000096145"

$`R-MMU-9026967`
[1] "ENSMUSG00000096145"

$`R-MMU-5619415`
[1] "ENSMUSG00000101174"

$`R-MMU-6807826`
[1] "ENSMUSG00000101959"

$`R-MMU-1655879`
[1] "ENSMUSG00000103711"

$`R-MMU-444419`
[1] "ENSMUSG00000104445"

$`R-MMU-446278`
[1] "ENSMUSG00000104445"

$`R-MMU-6786789`
[1] "ENSMUSG00000110206"

$`R-MMU-8854736`
[1] "ENSMUSG00000110206"

$`R-MMU-9604767`
[1] "ENSMUSG00000110206"

$`R-MMU-9706364`
[1] "ENSMUSG00000110206"

$`R-MMU-8850818`
[1] "ENSMUSG00000110576"

$`R-MMU-450971`
[1] "ENSMUSG0000011

In [10]:
%%R
rxns <- rxn2ensembls.nls %>% names()

In [11]:
%%R
rxn_knn_misclass_rate.nls <- list()
rxn_knn_ari.nls <- list()
rxn_knn_ecount.nls <- list()
rxn_pca.nls <- list()
count <- 0

In [12]:
%%R
toi_indices <- seq(1,length(tcdd_dose_detail.vec))
print(toi_indices)

 [1]  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25
[26] 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50
[51] 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75
[76] 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91


In [13]:
%%R
tcdd_dose_detail_vec_tis_of_interest <-
   tcdd_dose_detail.vec[toi_indices]
print(tcdd_dose_detail_vec_tis_of_interest)

 [1] 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00
[16] 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00
[31] 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00
[46] 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.01 0.01 0.01 0.01 0.01 0.01
[61] 0.01 0.01 0.01 0.01 0.00 0.00 0.00 0.01 0.00 0.00 0.00 0.00 0.01 0.00 0.01
[76] 0.01 0.01 0.01 0.01 0.00 0.01 0.01 0.01 0.00 0.00 0.00 0.00 0.00 0.01 0.01
[91] 0.01


In [14]:
%%R
vst.count.mtx.tis_of_interest <- vst.count.mtx[, toi_indices]
print(vst.count.mtx.tis_of_interest)

                   SRR7817611 SRR7817612 SRR7817614 SRR7817615 SRR7817616
ENSMUSG00000000001  15.595318  15.614104  15.551779  15.627745  15.601970
ENSMUSG00000000028   9.915932   9.776246  10.016167   9.866338   9.934504
ENSMUSG00000000049  19.743729  19.723414  19.974622  19.944275  19.947717
ENSMUSG00000000058  10.500222  10.636379  10.666665  10.388602  10.908824
ENSMUSG00000000085  12.209777  11.972662  12.116712  12.117091  11.772576
ENSMUSG00000000088  16.576893  16.582078  16.547906  16.265580  16.221265
ENSMUSG00000000120  11.565879  11.626400  11.673038  11.290948  11.230115
ENSMUSG00000000126   9.686067  10.025535   9.563343  10.073716   9.926025
ENSMUSG00000000127  11.511770  11.320054  11.603656  11.505857  11.755857
ENSMUSG00000000142  11.687984  11.668344  11.646364  12.385733  12.147609
ENSMUSG00000000149  15.840240  15.906962  15.872712  15.981036  15.777121
ENSMUSG00000000154  15.352727  15.437657  15.506350  15.558132  15.570458
ENSMUSG00000000168  14.675662  14.7026

ENSMUSG00000001105  14.229837  14.235340  13.880035  13.923989  13.863994
ENSMUSG00000001119  11.589388  11.073297  11.304662  11.403052  11.867436
ENSMUSG00000001123  18.013368  17.995907  18.132371  18.173662  18.197405
ENSMUSG00000001127  15.078380  15.113197  15.276463  15.325558  15.421168
ENSMUSG00000001128  13.180274  13.298749  12.697861  12.933558  12.921292
ENSMUSG00000001131   9.121350   9.117650   8.603026   8.619865   9.593587
ENSMUSG00000001143  12.728168  12.563328  12.890728  12.877593  12.648379
ENSMUSG00000001155  18.439903  18.379842  18.505059  18.355614  18.320334
ENSMUSG00000001158  12.205025  12.022704  12.411486  12.019850  12.004842
ENSMUSG00000001173  11.370665  11.402861  11.837390  11.403052  11.778875
ENSMUSG00000001175  16.991750  17.012846  17.077289  17.160159  17.195348
ENSMUSG00000001211  16.717995  16.717246  16.716137  16.733842  16.695280
ENSMUSG00000001225   8.620712   9.113146   8.603026   8.619865   8.619758
ENSMUSG00000001228   9.756620   9.9256

ENSMUSG00000002249  12.317308  12.337395  12.450556  12.520606  12.272385
ENSMUSG00000002250  14.521244  14.641092  14.179049  14.808889  14.879399
ENSMUSG00000002257   9.524942   9.409666   9.465951   9.750243   9.680771
ENSMUSG00000002279  13.676802  14.028160  14.012199  14.187063  14.089957
ENSMUSG00000002289  13.291273  13.146899  14.261702  13.075841  13.284779
ENSMUSG00000002297  10.538247  10.677589  10.379145  10.315201  10.192591
ENSMUSG00000002307  12.482211  12.495343  12.532797  12.636494  12.634694
ENSMUSG00000002308  11.249139  11.218355  10.919826  11.455751  11.160023
ENSMUSG00000002324  10.418648  10.192182  10.799830   9.869754  10.071765
ENSMUSG00000002325  14.016660  14.095138  14.025667  14.317916  14.431668
ENSMUSG00000002326  12.712031  12.734593  12.507269  12.564710  12.664864
ENSMUSG00000002346  15.872448  15.903449  15.766979  15.818283  15.848240
ENSMUSG00000002365  14.055599  14.093446  14.348108  14.317015  14.371222
ENSMUSG00000002379  14.500573  14.3147

ENSMUSG00000009281  17.510099  17.497812  17.713124  17.718975  17.568250
ENSMUSG00000009292  10.501158  10.930301  10.583651  10.609429  11.043623
ENSMUSG00000009293  13.318831  13.050788  13.592142  13.560053  13.256781
ENSMUSG00000009350   8.689773   8.689570   9.203985   8.642647   8.623471
ENSMUSG00000009356   8.689773   8.689570   8.642881   8.642647   8.623471
ENSMUSG00000009376  15.527531  15.509696  15.797653  15.752062  15.493488
ENSMUSG00000009394   8.689773   8.689570   8.642881   8.642647   8.623471
ENSMUSG00000009406  11.290375  11.533289  11.767440  11.727630  11.701348
ENSMUSG00000009470  15.335629  15.384903  15.321326  15.398705  15.427910
ENSMUSG00000009487   8.689773   8.689570   8.642881   8.642647   8.623471
ENSMUSG00000009535  12.698970  12.986040  12.523621  12.306502  12.796041
ENSMUSG00000009545   9.806114  10.071217   9.845982  10.127701   9.827017
ENSMUSG00000009549  13.252080  13.539768  13.851842  13.609958  13.884317
ENSMUSG00000009555  13.766115  13.4730

ENSMUSG00000013833  13.404144  13.521448  13.710835  13.761762  13.447140
ENSMUSG00000013846  14.836645  15.050242  15.043357  15.125236  15.273036
ENSMUSG00000013921   9.953064   9.804961   9.396546   9.198018   9.438967
ENSMUSG00000013936   8.689773   8.689570   8.642881   8.642647   8.623471
ENSMUSG00000013974   8.689773   8.689570   8.642881   8.642647   9.311407
ENSMUSG00000014030  10.168677  10.599803   9.203985   9.203110   9.537835
ENSMUSG00000014074  12.192782  12.395702  12.038910  12.063277  12.392415
ENSMUSG00000014075  10.557954   9.804961   9.403073   9.658246   9.902146
ENSMUSG00000014077  17.097613  17.124181  17.383209  17.390990  17.349104
ENSMUSG00000014158  10.079794   9.625556   9.925384  10.166401   9.305520
ENSMUSG00000014164  11.983940  11.901308  12.058759  11.836246  12.549522
ENSMUSG00000014232  11.102698  11.488716  11.633100  11.833102  11.961843
ENSMUSG00000014245  11.498549  11.687431  11.221541  11.308085  11.306557
ENSMUSG00000014294  13.981366  13.8664

...

ENSMUSG00000000776  12.058164  12.242667  11.771233  11.911531  11.637258
ENSMUSG00000000782  11.519650  11.453134  11.746353  11.717309  10.520990
ENSMUSG00000000787  16.037831  16.071799  16.842243  16.866280  16.604062
ENSMUSG00000000791   9.625691   9.527413   9.555325   9.503581   9.791720
ENSMUSG00000000792   8.611166   8.611235   8.626693   8.626104   8.628916
ENSMUSG00000000794   8.887954   8.611235   9.347000   9.453295   8.628916
ENSMUSG00000000805   8.611166   8.611235   8.626693   8.626104   8.628916
ENSMUSG00000000817   9.465650   9.539872   9.456502   9.441729   9.331045
ENSMUSG00000000823  14.615996  14.577561  14.592571  14.468783  14.135249
ENSMUSG00000000826  14.955897  14.983722  14.731187  14.808915  14.862401
ENSMUSG00000000869   9.106424   8.815111   8.834301   9.323835   8.628916
ENSMUSG00000000876  16.296817  16.252846  16.381235  16.451334  16.184009
ENSMUSG00000000881  12.806763  12.593282  12.897608  12.920075  12.953505
ENSMUSG00000000889   8.611166   8.6112

ENSMUSG00000001918  11.102404  11.211820  10.904413  11.044671  11.235713
ENSMUSG00000001924  16.690012  16.715281  16.426448  16.421591  16.328707
ENSMUSG00000001930  11.203284  11.233128  10.797955  11.176884  10.989716
ENSMUSG00000001946  12.335321  12.479142  12.083626  12.093150  12.174089
ENSMUSG00000001962  13.164618  12.926628  12.915529  12.960696  12.419584
ENSMUSG00000001964  12.999699  13.021693  12.671162  12.649093  12.487953
ENSMUSG00000001983  12.895484  12.984075  12.179859  12.271753  12.658037
ENSMUSG00000001985   8.611166   8.611235   8.626693   8.626104   8.628916
ENSMUSG00000001986  10.068308   9.740512  10.560625  10.306249   9.810235
ENSMUSG00000001998  10.970900  11.247411  12.014307  11.853750  11.725449
ENSMUSG00000001999  12.057533  11.931521  12.224802  12.329010  12.706185
ENSMUSG00000002010  14.635901  14.668741  14.765033  14.740226  14.404011
ENSMUSG00000002015  15.914207  15.921368  16.186217  16.050591  16.121675
ENSMUSG00000002020  10.267770  10.6679

ENSMUSG00000003053  19.614018  19.641186  19.440494  19.443483  20.088291
ENSMUSG00000003062  12.157108  12.037909  11.776997  12.127076  12.269916
ENSMUSG00000003068  16.477351  16.447029  15.860876  15.876714  15.026899
ENSMUSG00000003070   9.253299   9.372460   9.326616   8.626104   9.334088
ENSMUSG00000003072  17.681467  17.690244  16.946545  16.956145  15.951354
ENSMUSG00000003099  14.477245  14.574617  13.969771  14.096749  13.780567
ENSMUSG00000003119  12.475329  12.565565  13.036613  12.822244  12.935693
ENSMUSG00000003131  13.439722  13.464276  13.888756  13.774254  13.858590
ENSMUSG00000003135  12.890299  13.040495  12.732647  12.657663  12.845893
ENSMUSG00000003153  10.147854  10.033383   9.726921   9.646717  10.113243
ENSMUSG00000003161  13.227616  13.234420  13.577312  13.880884  13.504037
ENSMUSG00000003184  15.073709  15.175216  14.596442  14.811061  14.614456
ENSMUSG00000003200  13.370443  13.488264  12.727499  12.803843  12.169352
ENSMUSG00000003206   9.925591  10.2979

ENSMUSG00000004267   9.539424   9.274898   9.143636   9.323835   8.923408
ENSMUSG00000004268  14.753962  14.715462  14.775329  14.728034  14.574254
ENSMUSG00000004270  16.070959  16.047819  15.831504  15.905869  15.400730
ENSMUSG00000004285  14.193574  14.078658  14.153269  14.015555  13.902358
ENSMUSG00000004296   9.253299   8.611235   9.143636   9.141441   9.337110
ENSMUSG00000004317  11.545947  11.268946  11.712828  11.431663  11.616664
ENSMUSG00000004319  13.893770  14.027834  14.649032  14.764818  14.585186
ENSMUSG00000004328   9.085798   8.611235   8.626693   8.626104   9.334088
ENSMUSG00000004341   9.844494   9.745106   9.559187   9.780727   9.950246
ENSMUSG00000004344   9.085798   8.611235   9.053996   9.270330   8.628916
ENSMUSG00000004347   9.085798   9.451874   9.650608   9.628095   9.574581
ENSMUSG00000004356  12.016637  11.987733  12.549786  12.866226  13.009258
ENSMUSG00000004364  13.322403  13.470953  14.491873  14.501743  14.830154
ENSMUSG00000004366   8.611166   8.6112

ENSMUSG00000005397  13.129518  13.122129  13.092606  12.639327  12.493858
ENSMUSG00000005410  11.547081  11.714784  10.389814  10.717060  10.552342
ENSMUSG00000005413  14.337927  14.367701  13.204965  13.144661  13.945285
ENSMUSG00000005447  10.782769  11.012269  10.395965  10.743230  10.355113
ENSMUSG00000005465  10.540496  10.597223  10.104152  10.387698  10.161922
ENSMUSG00000005469  15.347084  15.267375  14.912477  14.946686  14.451432
ENSMUSG00000005474   8.611166   8.611235   8.626693   8.626104   8.628916
ENSMUSG00000005481  12.935278  12.997195  12.492080  12.527476  12.678361
ENSMUSG00000005483  13.086287  13.138292  12.608387  12.576759  12.367290
ENSMUSG00000005493   9.085798   9.068593   8.626693   9.141441   8.628916
ENSMUSG00000005510  15.721480  15.650543  14.962263  15.047560  14.312798
ENSMUSG00000005533  10.704857  10.970052  11.054911  11.087535  10.531045
ENSMUSG00000005534  15.699584  15.726078  16.429544  16.518180  16.538427
ENSMUSG00000005547  17.040327  17.0696

In [15]:
%%R
training_indices <-
   caret::createDataPartition(
      tcdd_dose_detail_vec_tis_of_interest,
      times = 1,
      p = 1.0, # no data will be held out when set to "1.0"
      list = FALSE
   )
print(training_indices)

In [16]:
%%R
vst.count.mtx.train <-
   vst.count.mtx.tis_of_interest[, training_indices] #9/10ths of data
vst.count.mtx.test  <-
   vst.count.mtx.tis_of_interest[, -training_indices] #1/10th of data

print(vst.count.mtx.train)

  11.596184  11.605035  11.741690  11.650988  12.033914
ENSMUSG00000001962  13.593174  13.419807  13.699750  13.511621  13.269829
ENSMUSG00000001964  12.514896  12.569657  12.914854  12.706912  12.717706
ENSMUSG00000001983  13.167887  13.040320  12.924918  13.207833  13.155707
ENSMUSG00000001985   8.620712   8.619719   8.603026   9.118193   8.619758
ENSMUSG00000001986  10.035818  10.375073   9.214823   9.862908   9.800301
ENSMUSG00000001998  11.771031  11.464736  11.695479  11.761778  11.609584
ENSMUSG00000001999  12.253509  12.098821  12.245217  11.943528  12.091580
ENSMUSG00000002010  15.154692  15.152283  15.040796  15.358482  15.344731
ENSMUSG00000002015  15.804628  15.832279  16.157700  15.787543  15.795379
ENSMUSG00000002020  10.208359  10.412187   9.731804  10.591724  10.432147
ENSMUSG00000002028  13.557087  13.597191  13.679548  13.810766  13.644855
ENSMUSG00000002031  13.035882  13.071203  13.491508  13.290168  13.092528
ENSMUSG00000002033   9.902711   9.925686   9.495476  10.

ENSMUSG00000000561  13.415154  13.553547  13.620067  13.689555  13.651288
ENSMUSG00000000562   8.618825   8.618196   8.626852   8.625534   8.625351
ENSMUSG00000000563  16.450403  16.538792  16.687259  16.728982  16.690685
ENSMUSG00000000567  10.953771  10.776017  10.875877  10.588412  11.211370
ENSMUSG00000000568  14.556662  14.598384  14.323615  14.292489  14.201976
ENSMUSG00000000579   9.289472   9.594651   9.934050   9.770885  10.079519
ENSMUSG00000000581  14.601758  14.721529  14.493282  14.709281  14.429017
ENSMUSG00000000594  15.850762  15.932678  15.918965  15.988478  15.817917
ENSMUSG00000000605  14.948367  14.913429  14.835831  14.923061  14.769627
ENSMUSG00000000617   8.618825   8.618196   8.626852   8.625534   8.625351
ENSMUSG00000000627   8.618825   8.618196   8.626852   9.321145   9.320284
ENSMUSG00000000628  10.465757  10.612947  10.604621  10.077255  10.079519
ENSMUSG00000000673  18.609524  18.628478  18.440211  18.446489  18.637151
ENSMUSG00000000682  11.300254  11.3444

ENSMUSG00000001642  10.496121  10.210371  10.276818  10.028150  10.449416
ENSMUSG00000001656   8.618825   8.618196   8.626852   8.625534   8.625351
ENSMUSG00000001657   8.618825   8.618196   8.626852   8.625534   8.625351
ENSMUSG00000001663  16.911479  16.963453  17.088710  17.108510  17.144328
ENSMUSG00000001670  19.959547  19.958317  19.226632  19.202525  19.090151
ENSMUSG00000001729  14.549112  14.407757  14.453543  14.530013  14.231347
ENSMUSG00000001741  10.833899  10.979977  10.688128  10.519695  11.009876
ENSMUSG00000001750  14.754017  14.734295  14.361169  14.374337  14.436087
ENSMUSG00000001751  13.732694  13.785481  13.100228  13.318327  13.616823
ENSMUSG00000001755  16.167355  16.101604  16.053634  16.080641  16.168961
ENSMUSG00000001761  11.900041  11.639382  11.509639  11.547335  11.710598
ENSMUSG00000001767  12.004107  12.290116  12.723924  12.543188  12.766739
ENSMUSG00000001768  12.813387  12.840181  12.692186  12.775858  12.308117
ENSMUSG00000001773   8.618825   8.6181

ENSMUSG00000002803  12.368597  12.217829  12.575655  12.443156  12.555720
ENSMUSG00000002804  12.228783  12.098990  12.074310  12.112799  11.975686
ENSMUSG00000002814  11.759351  11.835622  11.654355  12.046730  11.793486
ENSMUSG00000002820  14.149897  14.131982  13.689355  13.956529  13.889600
ENSMUSG00000002846  13.705097  13.645531  13.719661  13.455719  13.493979
ENSMUSG00000002847  14.939936  14.862700  14.670873  14.587633  14.548435
ENSMUSG00000002870  10.871494  11.093041  10.820177  10.878779  11.158159
ENSMUSG00000002885  13.042584  13.293068  12.497502  12.716844  12.741174
ENSMUSG00000002900  11.423513  11.494701  11.663969  11.636859  12.120803
ENSMUSG00000002908  11.563191  11.643910  11.141027  11.494092  11.231329
ENSMUSG00000002944  14.723032  14.757001  13.226512  13.093417  13.265352
ENSMUSG00000002948  14.489633  14.262198  14.542622  14.532175  14.487964
ENSMUSG00000002957  15.246598  15.307795  15.603472  15.601744  15.768164
ENSMUSG00000002963  13.427924  13.5890

ENSMUSG00000003934   8.618825   9.410282   8.626852   9.139316   9.552987
ENSMUSG00000003970  16.936797  16.978451  17.254049  17.212247  17.241565
ENSMUSG00000003974   8.618825   8.618196   8.626852   8.625534   8.625351
ENSMUSG00000004018  12.023990  11.948530  11.499113  11.533998  12.018916
ENSMUSG00000004032  12.069511  12.130612  11.751074  11.809450  12.237004
ENSMUSG00000004035  15.218012  15.224660  15.012231  15.017897  15.207648
ENSMUSG00000004038  16.531208  16.595942  15.167020  15.052142  15.386739
ENSMUSG00000004040  16.310544  16.274343  16.118644  16.121763  15.690418
ENSMUSG00000004043  13.106760  12.768761  13.216546  12.942765  13.607368
ENSMUSG00000004044  11.661522  11.732314  10.983811  11.148943  11.497032
ENSMUSG00000004054  13.829180  13.952892  13.962688  14.193829  14.089153
ENSMUSG00000004056  15.189293  15.265164  15.121170  15.232149  15.287522
ENSMUSG00000004070  14.767087  14.784830  14.549309  14.652621  14.594744
ENSMUSG00000004096  14.189222  14.2785

...

ENSMUSG00000000787  17.125130  16.813226  16.889869  16.618803  16.727475
ENSMUSG00000000791   9.682623   9.945744  10.024927  10.168487   9.844659
ENSMUSG00000000792   8.630497   8.618447   8.618446   8.601542   8.610483
ENSMUSG00000000794   9.194032   9.177791   9.170270   8.967754   9.083256
ENSMUSG00000000805   8.630497   8.618447   8.618446   8.601542   8.610483
ENSMUSG00000000817   9.157815   8.618447   8.618446   9.205224   8.610483
ENSMUSG00000000823  14.075642  14.419198  14.398417  14.118132  14.252051
ENSMUSG00000000826  14.771104  15.222755  15.190090  14.961003  14.964703
ENSMUSG00000000869   9.341526   9.287690   9.411643   9.155600   9.329237
ENSMUSG00000000876  16.327903  17.232019  17.208562  16.223161  16.501806
ENSMUSG00000000881  13.028195  12.848955  12.875319  13.144542  13.084139
ENSMUSG00000000889   8.630497   8.618447   8.618446   8.601542   9.083256
ENSMUSG00000000901  10.564694  11.162428  11.134022  10.967600  10.742548
ENSMUSG00000000902  12.278358  12.5569

ENSMUSG00000001930  11.652219  11.756225  11.881339  11.491084  11.254310
ENSMUSG00000001946  12.112182  12.506714  12.195516  12.297404  12.362954
ENSMUSG00000001962  12.122479  12.368993  12.383788  12.678823  12.530710
ENSMUSG00000001964  12.137121  12.527126  12.727175  12.392090  12.505063
ENSMUSG00000001983  12.200058  12.897852  12.873393  12.707196  12.833968
ENSMUSG00000001985   8.630497   8.618447   8.618446   8.601542   8.610483
ENSMUSG00000001986  10.067991  10.014562  10.105152   9.577913   9.608699
ENSMUSG00000001998  12.196075  11.891354  11.894726  11.864278  11.848828
ENSMUSG00000001999  12.304769  12.344395  12.486208  12.519340  12.687921
ENSMUSG00000002010  14.362543  14.554571  14.357534  14.602574  14.621197
ENSMUSG00000002015  16.143121  16.067310  16.172632  16.319901  16.252175
ENSMUSG00000002020  10.302611  10.140957  10.371135  10.355694   9.813075
ENSMUSG00000002028  13.706405  13.789890  13.848405  13.495329  13.811667
ENSMUSG00000002031  13.986841  13.7091

ENSMUSG00000003068  14.892465  15.112889  15.132889  15.168668  15.026298
ENSMUSG00000003070   8.630497   9.411651   8.618446   8.601542   9.083256
ENSMUSG00000003072  15.454736  15.722629  15.666666  16.127926  15.869233
ENSMUSG00000003099  13.713546  14.043659  13.907235  14.008318  14.114609
ENSMUSG00000003119  13.249718  13.293354  13.162372  12.715096  13.098747
ENSMUSG00000003131  13.823335  14.112034  14.117047  13.764377  13.866600
ENSMUSG00000003135  13.058270  12.923805  12.891230  13.014712  12.788148
ENSMUSG00000003153   9.675787  10.224774  10.013599  10.142575   9.614139
ENSMUSG00000003161  13.815823  13.324948  13.384087  13.646490  13.406476
ENSMUSG00000003184  14.147628  14.746787  14.738151  14.826094  14.696412
ENSMUSG00000003200  12.586711  12.490874  12.596515  12.585306  12.733219
ENSMUSG00000003206   9.580235   9.846632  10.145122   9.767040   9.734614
ENSMUSG00000003208  11.154437  11.130363  10.495692  10.867103  11.141767
ENSMUSG00000003226  15.821765  15.3553

ENSMUSG00000004270  15.534165  15.407504  15.420646  15.477204  15.574467
ENSMUSG00000004285  13.580982  13.715532  13.638551  14.109259  13.937741
ENSMUSG00000004296   9.153033   9.411651   8.618446   9.050014   8.610483
ENSMUSG00000004317  11.701127  11.933538  11.928386  11.805059  11.938961
ENSMUSG00000004319  14.725764  15.074844  15.066369  14.633630  14.784826
ENSMUSG00000004328   9.679213   9.671513   9.739573   9.552992   9.368374
ENSMUSG00000004341  10.426144  10.121508  10.060869  10.257471  10.021822
ENSMUSG00000004344   9.157815   8.618447   8.618446   8.601542   9.359951
ENSMUSG00000004347   9.826417   9.675779   9.593104   9.610089   9.544135
ENSMUSG00000004356  13.049384  13.198582  13.224940  13.021016  13.056291
ENSMUSG00000004364  15.078933  14.717571  14.691517  14.630578  14.699062
ENSMUSG00000004366   8.630497   8.618447   8.618446   8.601542   8.610483
ENSMUSG00000004371   8.630497   8.618447   8.618446   9.207803   9.083256
ENSMUSG00000004383  11.509622  11.4367

ENSMUSG00000005413  12.763261  13.689385  13.793135  13.548745  12.915032
ENSMUSG00000005447  10.393143  10.678675  10.700277  10.832633  10.259201
ENSMUSG00000005465  10.222591  10.227100  10.222428  10.041676   9.984042
ENSMUSG00000005469  14.421667  14.773014  14.802456  14.458231  14.809032
ENSMUSG00000005474   8.630497   8.618447   8.618446   8.601542   9.083256
ENSMUSG00000005481  12.284430  12.560144  12.299677  13.075162  12.457388
ENSMUSG00000005483  12.200589  12.173006  12.196614  12.368234  12.466602
ENSMUSG00000005493   9.157815   8.618447   8.618446   8.601542   8.610483
ENSMUSG00000005510  13.851538  15.118494  15.143787  14.570344  14.490440
ENSMUSG00000005533  10.900216  10.923411  10.926634  11.078151  10.553306
ENSMUSG00000005534  16.769282  16.566202  16.657306  16.369880  16.420576
ENSMUSG00000005547  18.006301  17.660982  17.690206  18.243700  18.078746
ENSMUSG00000005553  10.479824  10.493845  10.262166  10.239314  10.601910
ENSMUSG00000005566  14.260307  14.4471

In [17]:
%%R
tcdd_dose_detail.vec.train <-
   tcdd_dose_detail_vec_tis_of_interest[training_indices]
tcdd_dose_detail.vec.test <-
   tcdd_dose_detail_vec_tis_of_interest[-training_indices]

# print(gtex_tissue_detail.vec.train)
print(length(unique(tcdd_dose_detail.vec.train)))

In [18]:
%%R
saveRDS(tcdd_dose_detail.vec.train,file=paste(OUT_DIR,"tcdd_dose_detail_vec_train0.01vs0_time_cross.Rds",sep=""))

In [19]:
%%R
cv_fold_indices <- caret::createFolds(tcdd_dose_detail.vec.train,
                                      k = N_FOLDS)
print(cv_fold_indices)

In [20]:
%%R
binary_tcdd_dose_annotations <- unique(tcdd_dose_detail.vec)
print(binary_tcdd_dose_annotations)

In [21]:
%%R
full_rxn_pca_results.nls <- list()
rxn_id_2_result_file_idx.nls <- list()

In [22]:
%%R
n_rxns <- length(rxns)
print(n_rxns)
result_idx <- 1
for(rxn_id_idx in seq(1:n_rxns)){
   rxn_id <- rxns[rxn_id_idx]
   rxn_pca <-
      prcomp(t(vst.count.mtx.train[rxn2ensembls.nls[[rxn_id]], ]), scale. = T)
   full_rxn_pca_results.nls[[rxn_id]] <- rxn_pca
   rxn_id_2_result_file_idx.nls[[rxn_id]] <- result_idx
   rxn_pca.nls[[rxn_id]] <-
      rxn_pca$x[, 1] # 1st principal component coordinate within this reaction-space for each sample
   if(mod(rxn_id_idx,100) == 0){
      print(paste("Processed ",rxn_id_idx,
                  " of ",n_rxns,
                  " reactions (",round((rxn_id_idx + 1)/n_rxns,digits = 3) * 100,"%)...",
                  sep=""))
      flush.console()
   }
   if(mod(rxn_id_idx,1000) == 0){
      print(paste("Storing PCA objects containing reactions ",rxn_id_idx-1000,
                  "-",rxn_id_idx,
                  " of ",n_rxns,
                  " reactions (",round((rxn_id_idx + 1)/n_rxns,digits = 3) * 100,"%)...",
                  sep=""))
      saveRDS(full_rxn_pca_results.nls,
              paste(OUT_DIR, "full_rxn_pca_results_nls",rxn_id_idx-1000,
                    "-",rxn_id_idx,"0.01vs0_time_cross.Rds", sep=""))
      full_rxn_pca_results.nls <- list()
      gc()
      result_idx <- result_idx + 1
   }
}

[1] "Processed 1100 of 9825 reactions (11.2%)..."
[1] "Processed 1200 of 9825 reactions (12.2%)..."
[1] "Processed 1300 of 9825 reactions (13.2%)..."
[1] "Processed 1400 of 9825 reactions (14.3%)..."
[1] "Processed 1500 of 9825 reactions (15.3%)..."
[1] "Processed 1600 of 9825 reactions (16.3%)..."
[1] "Processed 1700 of 9825 reactions (17.3%)..."
[1] "Processed 1800 of 9825 reactions (18.3%)..."
[1] "Processed 1900 of 9825 reactions (19.3%)..."
[1] "Processed 2000 of 9825 reactions (20.4%)..."
[1] "Storing PCA objects containing reactions 1000-2000 of 9825 reactions (20.4%)..."
[1] "Processed 2100 of 9825 reactions (21.4%)..."
[1] "Processed 2200 of 9825 reactions (22.4%)..."
[1] "Processed 2300 of 9825 reactions (23.4%)..."
[1] "Processed 2400 of 9825 reactions (24.4%)..."
[1] "Processed 2500 of 9825 reactions (25.5%)..."
[1] "Processed 2600 of 9825 reactions (26.5%)..."
[1] "Processed 2700 of 9825 reactions (27.5%)..."
[1] "Processed 2800 of 9825 reactions (28.5%)..."
[1] "Processed

In [23]:
%%R
# store remaining PCA objects and removing from RAM
saveRDS(rxn_id_2_result_file_idx.nls,
        paste(OUT_DIR,"rxn_id_2_result_file_idx_nls0.01vs0_time_cross.Rds",sep=""))
saveRDS(full_rxn_pca_results.nls,
        paste(OUT_DIR, "full_rxn_pca_results_nls0.01vs0_time_cross.Rds", sep=""))
rm(full_rxn_pca_results.nls)
gc()

           used  (Mb) gc trigger  (Mb) max used  (Mb)
Ncells  6950698 371.3   13075665 698.4 11898591 635.5
Vcells 15945283 121.7   25842636 197.2 25842634 197.2


In [24]:
%%R
# compare informaction content of below files with pca plots or similar
saveRDS(rxn_pca.nls, paste(OUT_DIR, "rxn_pca_nls0.01vs0_time_cross.Rds", sep = ""))
saveRDS(vst.count.mtx.train,
        paste(OUT_DIR, "vst_count_mtx_train0.01vs0_time_cross.Rds", sep = ""))

In [25]:
%%R
for (cv_fold in names(cv_fold_indices)) {
  cur_cv_fold_indices <- cv_fold_indices[[cv_fold]]
}

In [26]:
%%R
# print(tissue_annotation)
# print(binary_tcdd_dose_annotations)
# print(paste("Processing tissue annotation:", tissue_annotation))
# print(length(binary_tcdd_dose_detail_vec.test.cv_test_list))
# print(sum(tcdd_dose_detail.vec.train.cv_test == tissue_annotation))
# print(length(rxns))

# binary_tcdd_dose_annotations <- c(0.00, 30.00, 0.01, 0.03, 0.10, 0.30, 1.00, 3.00, 10.00)

formatted_annotations <- format(binary_tcdd_dose_annotations, nsmall = 2)

formatted_annotations <- trimws(formatted_annotations)

binary_tcdd_dose_annotations <- formatted_annotations

print(binary_tcdd_dose_annotations)
for (annotation in binary_tcdd_dose_annotations) {
    print(annotation)
}

[1] "0.00" "0.01"
[1] "0.00"
[1] "0.01"


In [27]:
%%R
# main loop
for (rxn_id_idx in seq(1:length(rxns))) {
   rxn_id <- rxns[rxn_id_idx]
   ensembl_ids <- rxn2ensembls.nls[[rxn_id]]
   
   mean_misclass_rate <- list()
   sum_ari <- 0

   for (cv_fold in names(cv_fold_indices)) {
      cur_cv_fold_indices <- cv_fold_indices[[cv_fold]]
      
      vst.count.mtx.train.cv_train <-
         vst.count.mtx.train[, -cur_cv_fold_indices] # 4/5ths of training features
      vst.count.mtx.train.cv_test <-
         vst.count.mtx.train[, cur_cv_fold_indices] # 1/5th of training features
      
      tcdd_dose_detail.vec.train.cv_train <-
         tcdd_dose_detail.vec.train[-cur_cv_fold_indices] # 4/5ths of training labels
      tcdd_dose_detail.vec.train.cv_test <-
         tcdd_dose_detail.vec.train[cur_cv_fold_indices] # 1/5th of training labels
      
#        print(length(vst.count.mtx.train.cv_train))
#        print(length(gtex_tissue_detail.vec.train.cv_train))
       
      binary_tcdd_dose_detail_vec.test.cv_test_list <- list()
      for (tissue_annotation in binary_tcdd_dose_annotations) {
#          print(tcdd_dose_detail.vec.train.cv_test)
#          print("-------------")
#          print(tissue_annotation)
#          print(binary_tcdd_dose_detail_vec.test.cv_test_list)
         binary_tcdd_dose_detail_vec.test.cv_test_list[[as.character(tissue_annotation)]] <-
            (as.character(tcdd_dose_detail.vec.train.cv_test) == as.character(tissue_annotation))
      }
      
      cv_train.expr_mat <-
         t(vst.count.mtx.train.cv_train[ensembl_ids, ])
      cv_test.expr_mat <-
         t(vst.count.mtx.train.cv_test[ensembl_ids, ])
#        print(dim(cv_train.expr_mat))
#        print(length(gtex_tissue_detail.vec.train.cv_train))
      
      rxn_knn_calls <- class::knn(train = cv_train.expr_mat,
                                  test = cv_test.expr_mat,
                                  cl = tcdd_dose_detail.vec.train.cv_train)
      
      # calculate & store adjusted rand index
      cur_ari <- pdfCluster::adj.rand.index(rxn_knn_calls,
                                            tcdd_dose_detail.vec.train.cv_test)
      sum_ari <- cur_ari + sum_ari
      
      # for each tissue, calculate misclassification rate
      for (tissue_annotation in binary_tcdd_dose_annotations) {
         cur_rxn_knn_calls <- (rxn_knn_calls == tissue_annotation)
            
         # calculate misclassification rate (https://stat.ethz.ch/pipermail/r-help/2011-September/288885.html)
         tab <- table(cur_rxn_knn_calls,
                      binary_tcdd_dose_detail_vec.test.cv_test_list[[tissue_annotation]])
        # print(tab)
         cur_misclass_rate <- 1 - sum(diag(tab)) / sum(tab)
        # print(paste("Misclass rate = 1 - ",sum(diag(tab))," / ",sum(tab)," = ", cur_misclass_rate,"...",sep=""))
         sum_misclass_rate <- cur_misclass_rate
         if (!is.null(mean_misclass_rate[[tissue_annotation]])) {
           sum_misclass_rate <- sum_misclass_rate + mean_misclass_rate[[tissue_annotation]]
         }
         mean_misclass_rate[[tissue_annotation]] <- sum_misclass_rate
      }
   }
   for(tissue_annotation in binary_tcdd_dose_annotations){
      mean_misclass_rate[[tissue_annotation]] <- (mean_misclass_rate[[tissue_annotation]] / N_FOLDS)
   }
   mean_ari <- sum_ari / N_FOLDS
   ecount <- length(ensembl_ids)
   
   rxn_knn_misclass_rate.nls[[rxn_id]] <- mean_misclass_rate
   assign("mean_misclass_rate",NULL,envir = .GlobalEnv)
   
   rxn_knn_ari.nls[[rxn_id]] <- mean_ari
   rxn_knn_ecount.nls[[rxn_id]] <- ecount
   
   count <- count + 1
   if (mod(count, 10) == 0) {
      print(
         paste(
            "Last RXN_ID = ",
            rxn_id,
            ": Last ARI = ",
            mean_ari,
            ": Last ECOUNT = ",
            ecount,
            ": Last Lung MISCLASS = ",
            rxn_knn_misclass_rate.nls[[rxn_id]][["Lung"]],
            ": Last Uterus MISCLASS = ",
            rxn_knn_misclass_rate.nls[[rxn_id]][["Uterus"]],
            ". Now ",
            round(1.0 - count / length(rxns),3) * 100,
            "% remaining..."
         )
      )
      flush.console()
   }
}

[1] "Last RXN_ID =  R-MMU-749456 : Last ARI =  0.757264957264957 : Last ECOUNT =  178 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  99.9 % remaining..."
[1] "Last RXN_ID =  R-MMU-9021601 : Last ARI =  0.44982296711904 : Last ECOUNT =  24 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  99.8 % remaining..."
[1] "Last RXN_ID =  R-MMU-3108203 : Last ARI =  0.764344603282656 : Last ECOUNT =  15 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  99.7 % remaining..."
[1] "Last RXN_ID =  R-MMU-9838627 : Last ARI =  0.44960880553337 : Last ECOUNT =  4 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  99.6 % remaining..."
[1] "Last RXN_ID =  R-MMU-193679 : Last ARI =  0.0816618464174461 : Last ECOUNT =  2 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  99.5 % remaining..."
[1] "Last RXN_ID =  R-MMU-194544 : Last ARI =  0.204736272384458 : Last ECOUNT =  2 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  99.4 % remaining..."
[1] "Last 

[1] "Last RXN_ID =  R-MMU-8949580 : Last ARI =  0.73260666170063 : Last ECOUNT =  18 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  94.9 % remaining..."
[1] "Last RXN_ID =  R-MMU-9692137 : Last ARI =  0.0563175841757937 : Last ECOUNT =  2 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  94.8 % remaining..."
[1] "Last RXN_ID =  R-MMU-450580 : Last ARI =  0.949295774647887 : Last ECOUNT =  12 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  94.7 % remaining..."
[1] "Last RXN_ID =  R-MMU-6791222 : Last ARI =  0.859214501510574 : Last ECOUNT =  61 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  94.6 % remaining..."
[1] "Last RXN_ID =  R-MMU-9696268 : Last ARI =  0.859214501510574 : Last ECOUNT =  41 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  94.5 % remaining..."
[1] "Last RXN_ID =  R-MMU-1912394 : Last ARI =  0.548755079728531 : Last ECOUNT =  9 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  94.4 % remaining..."
[1] "La

[1] "Last RXN_ID =  R-MMU-3928599 : Last ARI =  0.107114966658061 : Last ECOUNT =  3 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  89.9 % remaining..."
[1] "Last RXN_ID =  R-MMU-3928625 : Last ARI =  0.329578052469619 : Last ECOUNT =  17 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  89.8 % remaining..."
[1] "Last RXN_ID =  R-MMU-3928657 : Last ARI =  0.0460115666166079 : Last ECOUNT =  10 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  89.7 % remaining..."
[1] "Last RXN_ID =  R-MMU-9034539 : Last ARI =  0.00876788918946057 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  89.6 % remaining..."
[1] "Last RXN_ID =  R-MMU-983156 : Last ARI =  0.855555555555556 : Last ECOUNT =  242 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  89.5 % remaining..."
[1] "Last RXN_ID =  R-MMU-174119 : Last ARI =  0.742694473667925 : Last ECOUNT =  18 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  89.4 % remaining..."
[1] 

[1] "Last RXN_ID =  R-MMU-3000411 : Last ARI =  0.811111111111111 : Last ECOUNT =  33 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  84.9 % remaining..."
[1] "Last RXN_ID =  R-MMU-5228523 : Last ARI =  0.811111111111111 : Last ECOUNT =  33 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  84.8 % remaining..."
[1] "Last RXN_ID =  R-MMU-2395328 : Last ARI =  0.212649634741648 : Last ECOUNT =  5 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  84.7 % remaining..."
[1] "Last RXN_ID =  R-MMU-8852134 : Last ARI =  0.825925925925926 : Last ECOUNT =  6 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  84.6 % remaining..."
[1] "Last RXN_ID =  R-MMU-354124 : Last ARI =  0.633962374880084 : Last ECOUNT =  13 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  84.5 % remaining..."
[1] "Last RXN_ID =  R-MMU-429415 : Last ARI =  0.646801256745514 : Last ECOUNT =  13 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  84.4 % remaining..."
[1] "Las

[1] "Last RXN_ID =  R-MMU-5682377 : Last ARI =  0.850505050505051 : Last ECOUNT =  9 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  79.9 % remaining..."
[1] "Last RXN_ID =  R-MMU-2426496 : Last ARI =  0.719192317813954 : Last ECOUNT =  10 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  79.8 % remaining..."
[1] "Last RXN_ID =  R-NUL-2467092 : Last ARI =  0.674169696385003 : Last ECOUNT =  7 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  79.7 % remaining..."
[1] "Last RXN_ID =  R-MMU-168162 : Last ARI =  0.147514116336181 : Last ECOUNT =  6 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  79.6 % remaining..."
[1] "Last RXN_ID =  R-MMU-445079 : Last ARI =  0.550508686644908 : Last ECOUNT =  9 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  79.5 % remaining..."
[1] "Last RXN_ID =  R-MMU-8856808 : Last ARI =  0.811111111111111 : Last ECOUNT =  19 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  79.4 % remaining..."
[1] "Last 

...

[1] "Last RXN_ID =  R-MMU-437092 : Last ARI =  0.195262392983193 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  20 % remaining..."
[1] "Last RXN_ID =  R-MMU-173597 : Last ARI =  0.0337659687659262 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  19.9 % remaining..."
[1] "Last RXN_ID =  R-MMU-5663185 : Last ARI =  0.413421722773835 : Last ECOUNT =  3 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  19.8 % remaining..."
[1] "Last RXN_ID =  R-MMU-111285 : Last ARI =  0.105253634735647 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  19.7 % remaining..."
[1] "Last RXN_ID =  R-MMU-9758661 : Last ARI =  0.9 : Last ECOUNT =  9 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  19.6 % remaining..."
[1] "Last RXN_ID =  R-MMU-433089 : Last ARI =  0.313906117931067 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  19.5 % remaining..."
[1] "Last RXN_ID =  R-MMU-264

[1] "Last RXN_ID =  R-MMU-9724745 : Last ARI =  0.0862725104802025 : Last ECOUNT =  2 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  15 % remaining..."
[1] "Last RXN_ID =  R-MMU-9010872 : Last ARI =  0.0341531853845276 : Last ECOUNT =  2 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  14.9 % remaining..."
[1] "Last RXN_ID =  R-MMU-194641 : Last ARI =  0.325971097143452 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  14.8 % remaining..."
[1] "Last RXN_ID =  R-MMU-202939 : Last ARI =  0.0299240472157881 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  14.7 % remaining..."
[1] "Last RXN_ID =  R-MMU-9831009 : Last ARI =  0.0766982156966674 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  14.6 % remaining..."
[1] "Last RXN_ID =  R-MMU-9833697 : Last ARI =  0.171340472542318 : Last ECOUNT =  2 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  14.5 % remaining..."
[1] "Last 

[1] "Last RXN_ID =  R-MMU-110352 : Last ARI =  -0.0784838789718613 : Last ECOUNT =  2 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  10 % remaining..."
[1] "Last RXN_ID =  R-MMU-975916 : Last ARI =  0.165066141966593 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  9.9 % remaining..."
[1] "Last RXN_ID =  R-MMU-417858 : Last ARI =  0.05391053933805 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  9.8 % remaining..."
[1] "Last RXN_ID =  R-MMU-446214 : Last ARI =  -0.0326365352200961 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  9.7 % remaining..."
[1] "Last RXN_ID =  R-MMU-5578712 : Last ARI =  0.275212360152119 : Last ECOUNT =  2 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  9.6 % remaining..."
[1] "Last RXN_ID =  R-MMU-352052 : Last ARI =  0.347746138321834 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  9.5 % remaining..."
[1] "Last RXN_ID = 

[1] "Last RXN_ID =  R-MMU-965067 : Last ARI =  -0.0137448230650065 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  5 % remaining..."
[1] "Last RXN_ID =  R-MMU-451056 : Last ARI =  -0.0314270309232083 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  4.9 % remaining..."
[1] "Last RXN_ID =  R-MMU-1640164 : Last ARI =  0.243021354964759 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  4.8 % remaining..."
[1] "Last RXN_ID =  R-MMU-1482897 : Last ARI =  0.0279375091916032 : Last ECOUNT =  3 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  4.7 % remaining..."
[1] "Last RXN_ID =  R-MMU-5340195 : Last ARI =  -0.0434006084352628 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  4.6 % remaining..."
[1] "Last RXN_ID =  R-MMU-9708866 : Last ARI =  0.109802795230306 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  4.5 % remaining..."
[1] "Last RXN

[1] "Last RXN_ID =  R-MMU-6786789 : Last ARI =  0.171754901549543 : Last ECOUNT =  1 : Last Lung MISCLASS =   : Last Uterus MISCLASS =   . Now  0.1 % remaining..."


In [28]:
%%R
saveRDS(
   rxn_knn_misclass_rate.nls,
   paste(OUT_DIR, "toi_rxn_knn_misclass_rate_nls0.01vs0_time_cross.Rds", sep = "")
)
saveRDS(rxn_knn_ari.nls,
        paste(OUT_DIR, "toi_rxn_knn_ari_nls0.01vs0_time_cross.Rds", sep = ""))
saveRDS(rxn_knn_ecount.nls,
        paste(OUT_DIR, "toi_rxn_knn_ecount_nls0.01vs0_time_cross.Rds", sep = ""))

In [29]:
%%R
d <- data.frame(
   RXN_ID = names(rxn2ensembls.nls),
   MISCLASS = unlist(rxn_knn_misclass_rate.nls),
   ARI = unlist(rxn_knn_ari.nls),
   ECOUNT = unlist(rxn_knn_ecount.nls)
)
print(d)
saveRDS(d, paste(OUT_DIR, "toi_summary_df0.01vs0_time_cross.Rds", sep = ""))

                          RXN_ID   MISCLASS           ARI ECOUNT
R-MMU-170666.0.00   R-MMU-170666 0.00000000  0.4420723205     14
R-MMU-170666.0.01   R-MMU-170671 0.14361111  0.4420723205     14
R-MMU-170671.0.00   R-MMU-170674 0.00000000  0.4420723205     14
R-MMU-170671.0.01   R-MMU-170686 0.14361111  0.4420723205     14
R-MMU-170674.0.00   R-MMU-380073 0.00000000  0.7572649573    182
R-MMU-170674.0.01   R-MMU-392187 0.14361111  0.3936914494     22
R-MMU-170686.0.00   R-MMU-392195 0.00000000  0.3936914494     22
R-MMU-170686.0.01   R-MMU-392202 0.14361111  0.3936914494     22
R-MMU-380073.0.00   R-MMU-749454 0.00000000  0.7572649573    178
R-MMU-380073.0.01   R-MMU-749456 0.05694444  0.7572649573    178
R-MMU-392187.0.00   R-MMU-751001 0.00000000  0.3434331851     21
R-MMU-392187.0.01  R-MMU-8964229 0.17666667  0.1405223303      5
R-MMU-392195.0.00  R-MMU-8964242 0.00000000  0.1405223303      5
R-MMU-392195.0.01  R-MMU-8964329 0.17666667  0.1405223303      5
R-MMU-392202.0.00  R-MMU-

R-MMU-205060.0.01  R-MMU-5654169 0.24722222  0.4586355907     14
R-MMU-205099.0.00  R-MMU-5654222 0.00000000  0.3677460296     12
R-MMU-205099.0.01  R-MMU-5654224 0.19750000  0.3677460296     12
R-MMU-205112.0.00  R-MMU-5654392 0.00000000  0.5292336458     20
R-MMU-205112.0.01  R-MMU-5654397 0.12166667  0.3097361936     16
R-MMU-205117.0.00  R-MMU-5654399 0.00000000  0.3097361936     16
R-MMU-205117.0.01  R-MMU-5654402 0.23333333  0.5080952512     20
R-MMU-209532.0.00  R-MMU-5654404 0.00000000  0.4098904739     16
R-MMU-209532.0.01  R-MMU-5654406 0.38138889  0.2582377005     18
R-MMU-209566.0.00  R-MMU-5654407 0.00000000  0.4098904739     16
R-MMU-209566.0.01  R-MMU-5654408 0.05333333  0.2667494301     12
R-MMU-9012761.0.00 R-MMU-5654409 0.00000000  0.2667494301     12
R-MMU-9012761.0.01 R-MMU-5654413 0.23944444  0.7388664426     17
R-MMU-3247837.0.00 R-MMU-5654416 0.00000000  0.5822079282     15
R-MMU-3247837.0.01 R-MMU-5654418 0.16666667  0.3717957293     14
R-MMU-3247840.0.00 R-MMU-

R-MMU-5654169.0.01 R-MMU-9619682 0.15055556  0.1617898418      6
R-MMU-5654222.0.00 R-MMU-9010527 0.00000000  0.0455929294      3
R-MMU-5654222.0.01 R-MMU-9011075 0.19611111  0.2440914769      2
R-MMU-5654224.0.00 R-MMU-9011080 0.00000000  0.0879508877      2
R-MMU-5654224.0.01 R-MMU-9011082 0.19611111  0.0879508877      2
R-MMU-5654392.0.00 R-MMU-9011144 0.00000000 -0.0499229059      3
R-MMU-5654392.0.01 R-MMU-9011146 0.12166667 -0.0499229059      3
R-MMU-5654397.0.00 R-MMU-9753028 0.00000000  0.2929275851      5
R-MMU-5654397.0.01 R-MMU-9753126 0.19388889  0.2929275851      5
R-MMU-5654399.0.00  R-MMU-514604 0.00000000  0.1624969707      2
R-MMU-5654399.0.01  R-MMU-975389 0.19388889  0.0783883311      4
R-MMU-5654402.0.00 R-MMU-1169394 0.00000000  0.8111821675      5
R-MMU-5654402.0.01 R-MMU-1169398 0.12166667  0.6800682110      5
R-MMU-5654404.0.00 R-MMU-1169402 0.00000000  0.7898186505      6
R-MMU-5654404.0.01 R-MMU-1169403 0.16305556  0.6243134806      4
R-MMU-5654406.0.00 R-MMU-

R-MMU-5654653.0.01 R-MMU-2395801 0.16055556  0.2451154050     12
R-MMU-5654655.0.00 R-MMU-2396594 0.00000000  0.6722982033     19
R-MMU-5654655.0.01 R-MMU-2424480 0.12805556  0.5086222067     15
R-MMU-5654658.0.00 R-MMU-2424482 0.00000000  0.5086222067     15
R-MMU-5654658.0.01 R-MMU-2424484 0.12805556  0.5313222173     22
R-MMU-5654659.0.00 R-MMU-2424486 0.00000000  0.5313222173     22
R-MMU-5654659.0.01 R-MMU-2424487 0.13416667  0.5313222173     22
R-MMU-5654662.0.00  R-MMU-388814 0.00000000  0.1347501895      6
R-MMU-5654662.0.01  R-MMU-388829 0.13416667  0.4335903172      9
R-MMU-5654663.0.00  R-MMU-388831 0.00000000  0.3262987124      8
R-MMU-5654663.0.01  R-MMU-388832 0.13166667  0.4335476505      9
R-MMU-5654664.0.00  R-MMU-388833 0.00000000  0.6961761576     17
R-MMU-5654664.0.01  R-MMU-389158 0.14055556  0.4335476505      9
R-MMU-5654667.0.00  R-MMU-389348 0.00000000  0.1888508536      8
R-MMU-5654667.0.01  R-MMU-389350 0.14416667  0.2206335895      8
R-MMU-5654669.0.00  R-MMU

R-MMU-9619682.0.01 R-MMU-9690410 0.26166667  0.0708223606      2
R-MMU-9010527.0.00 R-MMU-9690416 0.00000000  0.0677745608      4
R-MMU-9010527.0.01 R-MMU-9690868 0.34055556  0.0020397590      3
R-MMU-9011075.0.00 R-MMU-9690898 0.00000000  0.0020397590      3
R-MMU-9011075.0.01 R-MMU-9692133 0.23500000  0.0521893831      2
R-MMU-9011080.0.00 R-MMU-9692136 0.00000000  0.0947689018      2
R-MMU-9011080.0.01 R-MMU-9692137 0.28222222  0.0563175842      2
R-MMU-9011082.0.00 R-MMU-9692138 0.00000000  0.0563175842      2
R-MMU-9011082.0.01 R-MMU-9692139 0.28222222  0.0521893831      2
R-MMU-9011144.0.00 R-MMU-9692145 0.00000000  0.0947689018      2
R-MMU-9011144.0.01 R-MMU-9692156 0.41000000  0.0268243189      4
R-MMU-9011146.0.00 R-MMU-9692157 0.00000000  0.0268243189      4
R-MMU-9011146.0.01 R-MMU-9692268 0.41000000  0.0659751153      2
R-MMU-9753028.0.00  R-MMU-450434 0.00000000 -0.0407247517      1
R-MMU-9753028.0.01  R-MMU-450466 0.19722222  0.8601769912     45
R-MMU-9753126.0.00  R-MMU

...

R-MMU-111883.0.00  R-MMU-9830451 0.00000000  0.3200477875      1
R-MMU-111883.0.01   R-MMU-975829 0.37305556  0.1748465724      1
R-MMU-111898.0.00   R-MMU-374787 0.00000000  0.1188502899      1
R-MMU-111898.0.01   R-MMU-373086 0.18527778  0.0731638781      5
R-MMU-1482759.0.00  R-MMU-211178 0.00000000  0.1111329254      1
R-MMU-1482759.0.01 R-MMU-9626928 0.37305556  0.1111329254      1
R-MMU-6791016.0.00 R-NUL-9626944 0.00000000  0.1111329254      1
R-MMU-6791016.0.01  R-MMU-170149 0.28472222  0.1343037090      1
R-MMU-9624789.0.00 R-MMU-5671737 0.00000000  0.2074551268      2
R-MMU-9624789.0.01 R-MMU-5671749 0.27361111  0.2074551268      2
R-MMU-5358592.0.00 R-MMU-9029987 0.00000000  0.1343037090      1
R-MMU-5358592.0.01 R-MMU-9733403 0.29277778  0.0462344727      1
R-MMU-69173.0.00    R-MMU-419426 0.00000000  0.1208166168      3
R-MMU-69173.0.01   R-MMU-9696323 0.29277778  0.1208166168      3
R-MMU-9014847.0.00  R-MMU-374214 0.00000000  0.1006570207      3
R-MMU-9014847.0.01  R-MMU

R-MMU-210282.0.00   R-MMU-428585 0.00000000  0.1358170096      1
R-MMU-210282.0.01  R-MMU-9844587 0.26000000  0.0032673826      1
R-MMU-1483165.0.00 R-MMU-1614544 0.00000000 -0.0733169217      1
R-MMU-1483165.0.01  R-MMU-742345 0.32916667 -0.0058854717      1
R-MMU-2167942.0.00  R-MMU-417820 0.00000000  0.1427824205      1
R-MMU-2167942.0.01 R-MMU-9707606 0.35500000  0.1505936132      1
R-MMU-8944261.0.00 R-MMU-9707683 0.00000000  0.1505936132      1
R-MMU-8944261.0.01 R-MMU-9707856 0.35500000  0.1505936132      1
R-MMU-2465917.0.00 R-MMU-1483087 0.00000000 -0.0561907317      1
R-MMU-2465917.0.01 R-MMU-1483211 0.40861111 -0.0561907317      1
R-MMU-2466085.0.00 R-MMU-1483219 0.00000000 -0.0561907317      1
R-MMU-2466085.0.01 R-MMU-1483229 0.40861111 -0.0561907317      1
R-MMU-9656891.0.00 R-MMU-5693691 0.00000000  0.2797874642      2
R-MMU-9656891.0.01 R-MMU-9749647 0.40861111 -0.0509476798      1
R-MMU-3341294.0.00 R-MMU-9749792 0.00000000  0.2797874642      2
R-MMU-3341294.0.01 R-MMU-

R-MMU-3446785.0.00 R-NUL-9025041 0.00000000  0.3748023804      1
R-MMU-3446785.0.01 R-NUL-9025059 0.19861111  0.3748023804      1
R-MMU-351947.0.00  R-NUL-9026445 0.00000000  0.3748023804      1
R-MMU-351947.0.01  R-NUL-9030427 0.33583333  0.3748023804      1
R-MMU-6810076.0.00 R-NUL-9033304 0.00000000  0.3748023804      1
R-MMU-6810076.0.01 R-NUL-9033333 0.36027778  0.3748023804      1
R-MMU-70577.0.00    R-MMU-443997 0.00000000  0.2402578141      1
R-MMU-70577.0.01    R-MMU-391168 0.16638889  0.1230998670      6
R-MMU-3204311.0.00 R-MMU-8849638 0.00000000 -0.0085220512      1
R-MMU-3204311.0.01 R-MMU-8849646 0.27333333 -0.0085220512      1
R-MMU-8944220.0.00 R-MMU-1362465 0.00000000  0.3272212143      1
R-MMU-8944220.0.01  R-MMU-112435 0.32138889  0.2700960119      2
R-NUL-2473498.0.00 R-MMU-9036987 0.00000000  0.0730406486      1
R-NUL-2473498.0.01 R-MMU-9603982 0.32138889  0.0730406486      1
R-MMU-159566.0.00  R-MMU-9603983 0.00000000  0.0730406486      1
R-MMU-159566.0.01   R-MMU

R-MMU-8956458.0.00 R-MMU-9832937 0.00000000  0.3159826827      2
R-MMU-8956458.0.01 R-MMU-9832947 0.32944444  0.3159826827      2
R-MMU-977389.0.00  R-MMU-5649764 0.00000000  0.1977333048     13
R-MMU-977389.0.01  R-MMU-5649799 0.29555556  0.1977333048     13
R-MMU-351210.0.00  R-MMU-5649801 0.00000000  0.1977333048     13
R-MMU-351210.0.01  R-MMU-8847424 0.42888889 -0.0417677369      1
R-MMU-9012721.0.00  R-MMU-561253 0.00000000  0.1932373165      1
R-MMU-9012721.0.01  R-MMU-428273 0.16027778 -0.0423564219      1
R-MMU-9013471.0.00 R-MMU-9836621 0.00000000 -0.0423564219      1
R-MMU-9013471.0.01  R-MMU-428779 0.16027778  0.2802486998      1
R-MMU-9013533.0.00 R-MMU-8987120 0.00000000  0.4313100811      1
R-MMU-9013533.0.01 R-MMU-8937442 0.16027778  0.7254520987      2
R-MMU-9034756.0.00 R-MMU-1254376 0.00000000  0.0531292311      1
R-MMU-9034756.0.01  R-MMU-211923 0.16027778  0.4206924920      1
R-MMU-3299682.0.00  R-MMU-189423 0.00000000  0.0592214037      1
R-MMU-3299682.0.01 R-MMU-

R-MMU-8954327.0.00 R-MMU-9020273 0.00000000  0.0237359850      1
R-MMU-8954327.0.01  R-MMU-350869 0.25583333  0.0614409188      1
R-MMU-159179.0.00  R-MMU-5696004 0.00000000 -0.0221396721      1
R-MMU-159179.0.01  R-MMU-9670578 0.21805556  0.1287282807      3
R-MMU-159194.0.00  R-MMU-8955010 0.00000000 -0.0159925199      1
R-MMU-159194.0.01    R-MMU-71825 0.21805556  0.3355856615      1
R-MMU-9756134.0.00 R-MMU-5696146 0.00000000  0.1203598053      1
R-MMU-9756134.0.01 R-MMU-6788810 0.21805556  0.2377088152      1
R-MMU-9756156.0.00 R-MMU-8944246 0.00000000  0.0063490436      1
R-MMU-9756156.0.01  R-MMU-420384 0.21805556  0.2867832367      1
R-MMU-9756183.0.00 R-MMU-6786239 0.00000000  0.1845873776      1
R-MMU-9756183.0.01 R-MMU-9717434 0.21805556  0.2056227910      1
R-MMU-9632038.0.00 R-MMU-5619427 0.00000000  0.2744805250      1
R-MMU-9632038.0.01 R-MMU-1605723 0.30694444  0.0514505775      1
R-MMU-9632039.0.00 R-MMU-4085029 0.00000000  0.0514505775      1
R-MMU-9632039.0.01 R-MMU-

In [30]:
%%R
pid <- read.table(paste(OUT_DIR,"tcdd_project_id0.01vs0_time_cross.txt",sep=""),header=FALSE)
sid <- read.table(paste(OUT_DIR,"tcdd_sample_id0.01vs0_time_cross.txt",sep=""), header=FALSE)
sex <- read.table(paste(OUT_DIR, "tcdd_gender0.01vs0_time_cross.txt", sep=""), header=FALSE)
print(sex)

       V1
1    male
2    male
3    male
4    male
5    male
6    male
7    male
8    male
9    male
10   male
11   male
12   male
13   male
14   male
15   male
16   male
17   male
18   male
19   male
20   male
21   male
22   male
23   male
24   male
25   male
26   male
27   male
28   male
29   male
30   male
31   male
32   male
33   male
34   male
35   male
36   male
37   male
38   male
39   male
40   male
41   male
42   male
43   male
44   male
45   male
46   male
47   male
48   male
49 female
50 female
51 female
52 female
53 female
54 female
55 female
56 female
57 female
58 female
59 female
60 female
61 female
62 female
63 female
64 female
65 female
66 female
67 female
68 female
69 female
70 female
71 female
72 female
73 female
74 female
75 female
76 female
77 female
78 female
79 female
80   male
81   male
82   male
83   male
84   male
85   male
86   male
87   male
88   male
89   male
90   male
91   male


In [31]:
%%R
library(htmlwidgets)
min_misclass_pca <-
   prcomp(t(vst.count.mtx.train), scale. = T)
pca.d <- data.frame(
   PC1 = min_misclass_pca$x[, 1],
   PC2 = min_misclass_pca$x[, 2],
   PC3 = min_misclass_pca$x[, 3],
   Section = tcdd_dose_detail.vec.train,
   Project_id = pid$V1,
   Sample_id = sid$V1
)

ggplot(pca.d, aes(x = PC1, y = PC2, colour = Section)) +
   geom_point(size = 3) +  # 绘制点，指定点的大小
   geom_text(aes(label = paste(Project_id, Sample_id)), vjust = -1, hjust = 1, size = 2) +  # 添加标签
   scale_colour_manual(values = c("0" = "red", "0.01" = "green")) +  # 手动指定颜色
   theme_bw()

p <- plot_ly(
   data = pca.d,
   x = ~PC1,
   y = ~PC2,
   z = ~PC3,
   type = "scatter3d",
   mode = "markers",
   marker = list(
     color = as.factor(pca.d$Section),  # 将 Section 转换为因子
     colors = c("red", "green"),  # 定义颜色
     size = 5
   ),
   text = ~paste("Project ID:", Project_id, "Sample ID:", Sample_id)  # 在悬停文本中显示 Project ID
)
saveWidget(p, "plotly_chart0.01vs0_time_cross.html", selfcontained = TRUE)